In [2]:
import os
current_directory = str(os.getcwd())

In [4]:
import numpy as np
from sympy import *
from fractions import Fraction
import pickle
from scipy.interpolate import griddata
import pandas as pd


#converting from 2nd unit to 1st
pc_kpc = 1e3  # number of pc in one kpc
cm_km = 1e5  # number of cm in one km
s_day = 24*3600  # number of seconds in one day
s_min = 60  # number of seconds in one hour
s_hr = 3600  # number of seconds in one hour
cm_Rsun = 6.957e10  # solar radius in cm
g_Msun = 1.989e33  # solar mass in g
cgs_G = 6.674e-8
cms_c = 2.998e10
g_mH = 1.6736e-24
g_me = 9.10938e-28
cgs_h = 6.626e-27
deg_rad = 180e0/np.pi
arcmin_deg = 60e0
arcsec_deg = 3600e0
cm_kpc = 3.086e+21  # number of centimeters in one parsec
cm_pc = cm_kpc/1e+3
s_Myr = 1e+6*(365*24*60*60)  # megayears to seconds

#REQUIRED FUNCTIONS
###########################################################################################################################################
#importing data from csv file to np array
def file_reader(list_of_file_names):
    dataframe_list=[]
    for i in list_of_file_names:
        df = pd.read_csv(current_directory+r'\{}.dat'.format(i),delimiter=',')
        dataframe_list.append(df)
    return dataframe_list

#extrapolation
def interpolation(list1,list2,standard):
    interpolated_data = griddata(list1, list2, standard, method='linear', fill_value=nan, rescale=False)
    return interpolated_data
###########################################################################################################################################
os.chdir(current_directory+'/M31_data')
raw_data = pd.read_csv('combined_data_m31.csv', skiprows=1)
os.chdir(current_directory)

In [40]:

def find_and_multiply_column(dataframe, substring, multiplier):
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = dataframe.copy()
    i = 0
    for col in result_df.columns:
        if substring in col:
            try:
                result_df[col] = result_df[col] * multiplier[i]
            except:
                result_df[col] = result_df[col] * multiplier
            i+=1
    return result_df

def keep_substring_columns(dataframe, substring):
    # Get the columns that contain the specified substring
    filtered_columns = [col for col in dataframe.columns if substring in col]
    
    # Create a new DataFrame with only the filtered columns
    result_df = dataframe[filtered_columns].copy()
    
    return result_df

radii_df = keep_substring_columns(raw_data, 'r ')
radii_df = radii_df.drop(columns='error kms')

#to obtain radius data from every df
distance_m31= 0.78 #Mpc
distances_Mpc=np.array([0.785,0.785,0.780,0.780])

#convert arcmin to kpc
radii_df = find_and_multiply_column(radii_df, 'r arcmin', ((distance_m31*1000)/(arcmin_deg*deg_rad)))
#convert arcsec to kpc
radii_df = find_and_multiply_column(radii_df, 'r arcsec', ((distance_m31*1000)/(arcsec_deg*deg_rad)))
#distance correction
radii_df = find_and_multiply_column(radii_df, 'r kpc', distance_m31/distances_Mpc)

# print("Original DataFrame:")
#print(raw_data)
print("\nDataFrame after multiplication:")
radii_df


DataFrame after multiplication:


,r kpc,r arcmin,r kpc.1,r arcmin.1,r kpc.2,r kpc.3,r arcmin.2,r arcsec
0,0.000000,0.378911,0.000000,0.378911,0.00,6.25,1.295558,0.378155
1,0.476943,0.755553,0.476943,0.755553,0.48,6.75,1.530165,0.756309
2,0.953885,1.134464,0.953885,1.134464,0.96,7.25,1.749570,1.134464
3,1.430828,1.513375,1.430828,1.513375,1.44,7.75,1.973514,1.512619
4,1.907771,1.890017,1.907771,1.890017,1.92,8.25,2.198137,1.890773
...,...,...,...,...,...,...,...,...
95,45.309554,36.302848,45.309554,36.302848,45.60,NaN,NaN,36.302848
96,NaN,36.681759,NaN,36.681759,NaN,NaN,NaN,36.681003
97,NaN,37.058401,NaN,37.058401,NaN,NaN,NaN,37.059158
98,NaN,37.437312,NaN,37.437312,NaN,NaN,NaN,37.437312


In [41]:
raw_data[radii_df.columns] = radii_df

In [42]:
raw_data

,r kpc,sigma_tot,r arcmin,sigma_HI,r kpc.1,sigma_HI.1,r arcmin.1,vcirc kms,error kms,r kpc.2,vcirc kms.1,r kpc.3,sigma_sfr,r arcmin.2,molfrac,r arcsec,vdisp kms no_warp,vdisp kms warp
0,0.000000,7652.0,0.378911,2.77,0.000000,0.001212,0.378911,0.0,0.0,0.00,0.00,6.25,0.40,1.295558,0.1084,0.378155,17.10,12.05
1,0.476943,4847.0,0.755553,3.19,0.476943,0.197500,0.755553,0.0,0.0,0.48,150.97,6.75,0.43,1.530165,0.3376,0.756309,8.58,15.64
2,0.953885,3022.0,1.134464,4.21,0.953885,0.394900,1.134464,336.2,171.7,0.96,221.65,7.25,0.52,1.749570,0.1946,1.134464,12.94,15.70
3,1.430828,1854.0,1.513375,3.94,1.430828,0.592400,1.513375,324.6,125.1,1.44,247.13,7.75,0.41,1.973514,0.1561,1.512619,17.12,10.75
4,1.907771,1275.0,1.890017,4.49,1.907771,0.789900,1.890017,339.0,52.8,1.92,252.51,8.25,0.43,2.198137,0.1423,1.890773,12.99,8.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,45.309554,0.0,36.302848,0.02,45.309554,0.000000,36.302848,255.0,5.8,45.60,210.85,NaN,NaN,NaN,NaN,36.302848,11.38,4.45
96,NaN,NaN,36.681759,0.03,NaN,NaN,36.681759,271.1,7.8,NaN,NaN,NaN,NaN,NaN,NaN,36.681003,6.81,4.40
97,NaN,NaN,37.058401,0.03,NaN,NaN,37.058401,269.8,4.7,NaN,NaN,NaN,NaN,NaN,NaN,37.059158,4.58,4.53
98,NaN,NaN,37.437312,0.03,NaN,NaN,37.437312,258.2,10.7,NaN,NaN,NaN,NaN,NaN,NaN,37.437312,9.05,5.62
